In [1]:
import os
import pprint
import math
import time
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../tfmodels')


import random
import numpy as np
from past.builtins import xrange
import pandas as pd

#from data import read_data
from memory_model import *
from tf_object import *


In [2]:
flags = tf.app.flags

flags.DEFINE_integer("nb_words", 100001, "term number in input sequence(zero mask)")
flags.DEFINE_integer("mem_size", 400, "the memory length of input sequence")
flags.DEFINE_integer("nhop", 6, "the number of hop layers")
flags.DEFINE_integer("embedding_size", 100, "word embedding size")
flags.DEFINE_float("init_hid", 0.1, "init_hid for q")
flags.DEFINE_float("init_std", 0.05, "init_std for training variable")

flags.DEFINE_float("linear_ratio", 0.5, "keep probability of drop out [0.9]")
flags.DEFINE_float("learning_rate", 0.01, "learning rate [0.001]")
flags.DEFINE_integer("batch_size", 1024, "batch size to use during training [128]")
flags.DEFINE_float("clip_gradients", 5.0, "clip gradients to this norm")
flags.DEFINE_integer("n_epochs", 100, "number of epoch to use during training")
flags.DEFINE_boolean("epoch_save", False, "save checkpoint or not in each epoch")
flags.DEFINE_integer("print_step", 500, "print step duraing training [100]")
flags.DEFINE_string("logs_dir", "logs/", "logs directory [logs/]")
flags.DEFINE_string("model_dir", "model/", "model directory [model/]")
flags.DEFINE_boolean("dir_clear", False, "clear the log and model directory")
flags.DEFINE_boolean("lr_annealing", True, "use lr annealing or not after each epoch [False]")
#flags.DEFINE_string("current_task_name", '_season_prediction', "current task name [self_prediction]")
flags.DEFINE_integer("gpu_id", 0, "default gpu id [0]")
flags.DEFINE_integer("gpu_num", 4, "gpu_num")

FLAGS = flags.FLAGS

In [3]:
#load training h5 file
#h5_file_path = '../RData/episode_seq_30d.h5'
#h5_file_path = 'Data/episode_seq_28_35.h5'
#h5_file_path = '/home/host/CORP/wei.wang/Data/episode_seq_30d.h5'   ##elscvgpu01
h5_file_path = '/mnt/host/storage/wei.wang/Data/episode_seq_30d.h5'  #elscvgpu02
h5f = h5py.File(h5_file_path, 'r')
videos = np.array(h5f['videos'])
viewthru = np.array(h5f['viewthru'])
hour_ids = np.array(h5f['hour_ids'])
#seasons = np.array(h5f['seasons'])
seq_len = h5f['seq_len']
seq_len = np.array(seq_len)
cancel_days = np.array(h5f['cancel_days'])
split_index = np.array(h5f['split_index'])
user_id = np.array(h5f['user_id'])

#load the user attributes
user_attr_path = '../RData/user_attr.csv'
user_attr_df = pd.read_csv(user_attr_path)
user_attr_dict = dict(zip(user_attr_df['userid'].tolist(), user_attr_df['id'].tolist()))
id_list = np.array([user_attr_dict[u] for u in user_id])


In [4]:
idxs = np.arange(0, len(cancel_days))
threshold = 60
cancel_days[cancel_days<=threshold] = 1
cancel_days[cancel_days>=threshold] = 0


In [ ]:
#using balance data in training stage
balance_flag = True
#training for debug&init parameter
training_flag = False

if balance_flag:
    train_cancel_days = cancel_days[:split_index]
    positive_length = sum(train_cancel_days==1)
    #[-positive_length:]
    train_idxs = np.append(np.random.choice(np.where(train_cancel_days == 0)[0], int(positive_length*3.0), replace=False),
                     np.random.choice(np.where(train_cancel_days == 1)[0], positive_length, replace=False))
else:
    #train_idxs = idxs[3142375:3303991]
    train_idxs = idxs[split_index[0]-500000:split_index[0]]
#print(len(train_idxs))

if training_flag:
    train_idxs, test_idxs = train_test_split(train_idxs, test_size=0.2, random_state=23)
    test_idxs = np.sort(test_idxs)
else:
    test_idxs = idxs[split_index:]



In [ ]:
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
with tf.Session(graph=graph_to_use, config=config) as session:
    #mem_model = MemN2NModel(FLAGS, session, 'memn2n_model_churn_prediction')
    mem_model = MemN2NUserModel(FLAGS, session, 'memn2n_user_model_churn_prediction')
    mem_model.build_model(type='churn', accK=1, nb_class=2)
    mem_model.build_model_summary()
    display(mem_model.model_summary())
    #we can init from a well-trained model by using model_restore()
    #mem_model.model_restore()
    #mem_model.run([videos,cancel_days], train_idxs, test_idxs)
    mem_model.run([videos,viewthru,id_list,cancel_days], train_idxs, test_idxs)

Initializing


,variable_name,variable_shape,parameters
0,global/Variable:0,[],1.0
1,gpu/memory/Ain_c/embedding_table:0,"[100000, 100]",10000000.0
2,gpu/memory/Ain_c_vt/embedding_table:0,"[100, 100]",10000.0
3,gpu/memory/Ain_t/W:0,"[400, 100]",40000.0
4,gpu/memory/questions/embedding_table:0,"[2000, 100]",200000.0
5,gpu/memory/Cin_c/embedding_table:0,"[100000, 100]",10000000.0
6,gpu/memory/Cin_c_vt/embedding_table:0,"[100, 100]",10000.0
7,gpu/memory/Cin_t/W:0,"[400, 100]",40000.0
8,gpu/momory_hops/hops_h/W:0,"[100, 100]",10000.0
9,gpu/prediction/fc/W:0,"[100, 2]",200.0


('Epoch', 1, '... training ...')
('Minibatch', 500, '/', 'loss:', 0.52771747)
('Minibatch', 500, '/', 'accuracy:', 0.75488281)
('epoch time:', 0.9475398341814677)
('Epoch', 1, 'training accuracy:', 0.74976573523723389)
('Epoch', 1, '... test ...')
('Epoch', 1, 'test accuracy:', 0.96181515200780365)
{'valid_los': 0.30948261477427452, 'loss': 0.54764589451003254, 'epoch': 0, 'valid_perplexity': 1.3627198805761858, 'best_auc': 0.67522586929033701, 'learning_rate': 0.01, 'best_accuracy': 0.74976573523723389, 'best_test_accuracy': 0.96181515200780365}
('Epoch', 2, '... training ...')
('Minibatch', 500, '/', 'loss:', 0.55668682)
('Minibatch', 500, '/', 'accuracy:', 0.73144531)
('epoch time:', 0.903827166557312)
('Epoch', 2, 'training accuracy:', 0.75000272400886936)
('Epoch', 2, '... test ...')
('Epoch', 2, 'test accuracy:', 0.96181515200780365)
{'valid_los': 0.30237564585534871, 'loss': 0.53882717117126933, 'epoch': 1, 'valid_perplexity': 1.3530694061701793, 'best_auc': 0.68638350969571527,

In [ ]:
"""
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
res = {}
with tf.Session(graph=graph_to_use, config=config) as session:
    mem_model = MemN2NUserModel(FLAGS, session, 'memn2n_user_model_churn_prediction')
    mem_model.build_model(type='churn', accK=1, nb_class=2)
    mem_model.build_model_summary()
    display(mem_model.model_summary())
    mem_model.model_restore()
    epochTestLoss, epochTestMetric, result = mem_model.model_run([videos,viewthru,id_list,cancel_days], test_idxs, run_type='self',
                                                                 mode='test', shuffle=False, save_metric=True)
    
    #for var in tf.trainable_variables():
    #    res[var.name] = var.eval()

result = np.concatenate(result)
y = np.array(cancel_days[test_idxs])[0:len(result)]
pred = result[:,1]
roc_auc_score(y,pred)
"""